TODO:
RETURN TOP 10 MATHCES
USE A BIGGER BOOKS DATASET

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

c:\Personal\Study Stuff\SynopsisBookSearchBackend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load
booksDataframe = pd.read_csv("./data/books_summary.csv")
booksDataframe = booksDataframe.drop_duplicates(subset="book_name", keep="first")
booksDataframe.to_csv("./cleaned_books.csv")

In [21]:
#preview
test_books = booksDataframe.head(1000)
print(test_books)

     Unnamed: 0                            book_name  \
0             0          The Highly Sensitive Person   
1             1  Why Has Nobody Told Me This Before?   
2             2                 The Midnight Library   
3             3                      Brave New World   
4             4                                 1984   
..          ...                                  ...   
995        1005                     The Power Of Bad   
996        1006           Who Will Cry When You Die?   
997        1007                      Super Attractor   
998        1008                       The Way Of Zen   
999        1009                    Everybody Matters   

                                             summaries categories  
0     is a self-assessment guide and how-to-live te...    science  
1     is a collection of a clinical psychologist’s ...    science  
2     tells the story of Nora, a depressed woman in...    science  
3     presents a futuristic society engineered perf... 

In [4]:
#load sbert
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [16]:
#create embeddings vector
summaries_embeddings = model.encode(test_books["summaries"], convert_to_tensor=True)

In [22]:
#input
user_input = input("Describe your preferred plot c:")
user_embedding = model.encode(user_input, convert_to_tensor=True)

In [45]:
#compute similarities and print top n results
similarity_scores = util.pytorch_cos_sim(user_embedding, summaries_embeddings)
similarity_scores = similarity_scores.cpu().numpy().flatten()

top_10_indices = np.argsort(similarity_scores)[::-1][:10]  # Sort & get last 10 in reverse

# Print the book titles and their similarity scores
for i, idx in enumerate(top_10_indices):
    print(f"Rank {i+1}: {test_books['book_name'][idx]} Summary: {test_books['summaries'][idx]} (Score: {similarity_scores[idx]:.4f})")
print(similarity_scores.shape)

Rank 1: AI 2041 Summary:  explores the concept of artificial intelligence and delves into some thought-provoking ideas about AI taking over the world in the next twenty years, from our day-to-day lives to our jobs, becoming a worldwide used tool that will shake the world as we know it from the ground up. (Score: 0.6398)
Rank 2: AI Superpowers Summary:  will help you understand what to expect of the effect that artificial intelligence will have on your future job opportunities by diving into where China and the US, the world’s two leaders in AI, are heading with this breakthrough technology. (Score: 0.6041)
Rank 3: Abundance Summary:  shows you the key technological trends being developed today, to give you a glimpse of a future that’s a lot brighter than you think and help you embrace the optimism we need to make it happen. (Score: 0.4935)
Rank 4: The Singularity Is Near Summary:  outlines the future of technology by describing how change keeps accelerating, what computers will look li